## Evolutionary origin

Macaca - Mouse - Human

In [1]:
import os,re,glob
import pandas as pd
import numpy as np
from collections import Counter
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

users_dir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction"
GENOMEDIR = "/genomics/users/marta/genomes"
species=["mouse","macaca","platypus","chicken","opossum"]

## annotation file
annotation="/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed/gencode.v38.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.gtf"
transcript_gene=pd.read_csv("/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed/1transcript_1gene.reconstructed.csv")

## evoDir
evoDir=os.path.join(users_dir,"EvolutionaryOrigin_MMseqs")

def translate_dna_to_protein(dna_seq):
    return str(Seq(dna_seq).translate())

# Function to save DF in fasta format
def create_seqrecord_PROT(row):
    return SeqRecord(Seq(row['protein']), id=row['header'][:-4], description="")
def create_seqrecord_DNA(row):
    return SeqRecord(Seq(row['seq']), id=row['header'], description="")

species_and_years_dict = {"specie" : ["human","macaca","mouse","opossum","platypus","chicken"],
                          "age" : [0,25,90,180,200,310]}
species_and_years = pd.DataFrame(data=species_and_years_dict, index=[1,2,3,4,5,6])
species_and_years


,specie,age
1,human,0
2,macaca,25
3,mouse,90
4,opossum,180
5,platypus,200
6,chicken,310


In [ ]:
## get repre sequences - both DNA and PROTEINS
for s in species:
    print(s)
    riboseq_dir = os.path.join(users_dir,s,"RiboSeq/RiboQC_RiboNovel")

    ## get all candidates
    print("Reading all the candidate ORF sequences")
    df = pd.read_csv(os.path.join(riboseq_dir,"Annotation/candidateORF.fa"), header=None, sep="\t")
    ## convert to df
    candidates_fasta = pd.DataFrame({'header':df[0].iloc[::2].values, 'seq':df[0].iloc[1::2].values})
    candidates_fasta['header'] = candidates_fasta['header'].str[1:]
    candidates_fasta = candidates_fasta[~candidates_fasta['header'].str.contains("iORF")]

    all_repre_candidates = pd.DataFrame()
    for sample in os.listdir(os.path.join(riboseq_dir,"RibORF")):
        if sample.endswith("r1"):
            print(sample)

            repre = pd.read_csv(os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.txt"), sep="\t")
            repre_fasta = candidates_fasta[candidates_fasta['header'].isin(repre.orfID.values.tolist())]
            repre_fasta['header'] = s + "_" +repre_fasta['header']

            ## keep only the coding and the non-coding
            repre_fasta = repre_fasta[repre_fasta['header'].str.contains("canonical")| repre_fasta['header'].str.contains("noncoding")]

            repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)
            all_repre_candidates = pd.concat([all_repre_candidates, repre_fasta])

            ## translate to protein
            print("Translating to protein")
            out_proteins = os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.PROTEIN.fa")
            seq_records_proteins = repre_fasta.apply(create_seqrecord_PROT, axis=1).tolist()
            SeqIO.write(seq_records_proteins, out_proteins, "fasta")

            ## keep the dna sequences as well
            print("Saving the original DNA ORF")
            out_dna = os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.DNA.fa")
            seq_records_dna = repre_fasta.apply(create_seqrecord_DNA, axis=1).tolist()
            SeqIO.write(seq_records_dna, out_dna, "fasta")

            ## get only the nonredundant
            print("Getting the non-redundant")
            all_repre_candidates.drop_duplicates(inplace=True)
            out_nonredundant_proteins = os.path.join(riboseq_dir,"RibORF")+"/"+s+"_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa"
            seq_records_proteins = all_repre_candidates.apply(create_seqrecord_PROT, axis=1).tolist()
            SeqIO.write(seq_records_proteins, out_nonredundant_proteins, "fasta")

## MMseqs2

Against all expressed in mouse, macaca, platypus, opossum in any tissue (non-redunant set)

#### TESTIS-EXPRESSED

In [4]:
## Testis-expressed ORFs human
TestisExpressed_translatedINtestis = pd.read_csv("/projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Q1_TestisORFs/human/ribORF_humanTestis_in1.csv")
TestisExpressed_translatedINtestis

def create_seqrecord_PROThuman(row):
    return SeqRecord(Seq(row['ORFpep']), id=row['orfID'], description="")

out_nonredundant_proteins = "/projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa"
seq_records_proteins = TestisExpressed_translatedINtestis.apply(create_seqrecord_PROThuman, axis=1).tolist()
SeqIO.write(seq_records_proteins, out_nonredundant_proteins, "fasta")

15040

Creating database of human (expressed in testis)

In [3]:
%%bash -s "$evoDir" "$users_dir"

# mkdir $1/MMseqsDB

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

mmseqs createdb /projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa $1/MMseqsDB/ribORF_humanTestis_in1_DB


/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin_MMseqs/MMseqsDB/ribORF_humanTestis_in1_DB exists and will be overwritten
createdb /projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin_MMseqs/MMseqsDB/ribORF_humanTestis_in1_DB 

MMseqs Version:       	14.7e284
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[=
Time for merging to ribORF_humanTestis_in1_DB_h: 0h 0m 1s 432ms
Time for merging to ribORF_humanTestis_in1_DB: 0h 0m 0s 874ms
Database type: Aminoacid
Time for processing: 0h 0m 6s 137ms


Creating a joint database with all the specie, everything translated, no matter the tissue

In [4]:
%%bash -s "$evoDir" "$users_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

# species="platypus mouse macaca chicken"
# for s in $species; do
#     echo $s

#     ### create DB
#     mmseqs createdb $2/$s/RiboSeq/RiboQC_RiboNovel/RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa $1/MMseqsDB/${s}_noredunantProts_DB

# done

mmseqs createdb $2/*/RiboSeq/RiboQC_RiboNovel*/RibORF/*_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa $1/MMseqsDB/speciesDB



/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin_MMseqs/MMseqsDB/speciesDB exists and will be overwritten
createdb /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/chicken/RiboSeq/RiboQC_RiboNovel/RibORF/chicken_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/macaca/RiboSeq/RiboQC_RiboNovel/RibORF/macaca_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/mouse/RiboSeq/RiboQC_RiboNovel/RibORF/mouse_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/opossum/RiboSeq/RiboQC_RiboNovel/RibORF/opossum_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/platypus/RiboSe

Creating a joint database with all the specie AND human for clustering, everything translated for the species, no matter the tissue, but only translated in 1 for testis

In [5]:
%%bash -s "$evoDir" "$users_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

mmseqs createdb $2/*/RiboSeq/RiboQC_RiboNovel*/RibORF/*_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/Q1_TestisORFs/human/ribORF_humanTestis_in1.fa $1/MMseqsDB/species_and_humanDB



/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin_MMseqs/MMseqsDB/species_and_humanDB exists and will be overwritten
createdb /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/chicken/RiboSeq/RiboQC_RiboNovel/RibORF/chicken_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/macaca/RiboSeq/RiboQC_RiboNovel/RibORF/macaca_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/mouse/RiboSeq/RiboQC_RiboNovel/RibORF/mouse_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/opossum/RiboSeq/RiboQC_RiboNovel/RibORF/opossum_repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/platy

Searching

In [6]:
%%bash -s "$evoDir" "$users_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2
mkdir -p $1/Searching

rm $1/Searching/resultDB*
## The alignment consists of two steps the prefilter and alignment. To run the search, type:
mmseqs search -a -s 6 $1/MMseqsDB/ribORF_humanTestis_in1_DB $1/MMseqsDB/speciesDB $1/Searching/resultDB $1/tmp

# Then, convert the result database into a BLAST tab formatted file (option -m 8 in legacy blast, -outfmt 6 in blast+):
# Format alignment output	query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits

mmseqs convertalis $1/MMseqsDB/ribORF_humanTestis_in1_DB $1/MMseqsDB/speciesDB $1/Searching/resultDB $1/Searching/resultDB.m8

search -a -s 6 /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin_MMseqs/MMseqsDB/ribORF_humanTestis_in1_DB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin_MMseqs/MMseqsDB/speciesDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin_MMseqs/Searching/resultDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin_MMseqs/tmp 

MMseqs Version:                        	14.7e284
Substitution matrix                    	aa:blosum62.out,nucl:nucleotide.out
Add backtrace                          	true
Alignment mode                         	2
Alignment mode                         	0
Allow wrapped scoring                  	false
E-value threshold                      	0.001
Seq. id. threshold                     	0
Min alignment length                   	0
Seq. id. mode                          	0
Altern

Clustering

In [8]:
%%bash -s "$evoDir" "$users_dir"

module load Miniconda3/4.9.2
source /soft/system/software/Miniconda3/4.9.2/bin/activate MMseqs2

mkdir -p $1/Clustering

rm -r $1/Clustering/species_and_human_cluDB*

## cluster all species and human
mmseqs cluster -c 0.5 --min-seq-id 0.5 $1/MMseqsDB/species_and_humanDB $1/Clustering/species_and_human_cluDB $1/tmp
mmseqs createtsv $1/MMseqsDB/species_and_humanDB $1/MMseqsDB/species_and_humanDB $1/Clustering/species_and_human_cluDB $1/Clustering/species_and_human_cluDB.tsv

cluster -c 0.5 --min-seq-id 0.5 /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin_MMseqs/MMseqsDB/species_and_humanDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin_MMseqs/Clustering/species_and_human_cluDB /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin_MMseqs/tmp 

MMseqs Version:                     	14.7e284
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit            

## Analysis

In [5]:
clustering = pd.read_csv(os.path.join(evoDir,"Clustering/species_and_human_cluDB.tsv"), header=None, sep="\t")
clustering = clustering[clustering[0].str.startswith("ENST")]
print("all human")
print(len(clustering))
clustering['transcript_id'] = clustering[0].str.split(":", expand=True)[0]
clustering['transcript_id'] = clustering[0].str.split(".", expand=True)[0]

clustering = clustering.merge(transcript_gene, on="transcript_id")
print("all human after annotation")
print(len(clustering))
clustering = clustering[["gene_name","gene_type",0,1]]

## non-coding
noncoding = clustering[clustering[0].str.contains("noncoding")]
noncoding.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_speciesDB.tsv"), header=None, index=None, sep="\t")
print("noncoding all")
print(len(noncoding))
print(len(set(noncoding[0].values.tolist())))

noncoding_conserved = noncoding[~noncoding[1].str.startswith("ENST")]
noncoding_conserved.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_Conserved.tsv"), header=None, index=None, sep="\t")
print("Conserved noncoding human ORFs")
print(len(set(noncoding_conserved[0].values.tolist())))
noncoding_conserved.columns = ['gene_name','gene_type','orfID',1]

noncoding_NonConserved = noncoding[noncoding[1].str.startswith("ENST")]
noncoding_NonConserved.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_NonConserved.tsv"), header=None, index=None, sep="\t")
print("Non-Conserved noncoding human ORFs")
print(len(set(noncoding_NonConserved[0].values.tolist())))
noncoding_NonConserved.columns = ['gene_name','gene_type','orfID',1]


## canonical
canonical = clustering[clustering[0].str.contains("canonical")]
print("Canonical all")
print(len(canonical))
print(len(set(canonical[0].values.tolist())))
canonical.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_speciesDB.tsv"), header=None, index=None, sep="\t")

canonical_conserved = canonical[~canonical[1].str.startswith("ENST")]
canonical_conserved.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_Conserved.tsv"), header=None, index=None, sep="\t")
print("Conserved canonical human ORFs")
print(len(set(canonical_conserved[0].values.tolist())))
canonical_conserved.columns = ['gene_name','gene_type','orfID',1]

canonical_conserved_noncoding = canonical_conserved[~canonical_conserved[1].str.contains("noncoding")]
canonical_conserved_noncoding.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_ConservedAsNoncoding.tsv"), header=None, index=None, sep="\t")
print("Conserved canonical human ORFs as noncoding in other species")
print(len(set(canonical_conserved_noncoding['orfID'].values.tolist())))

canonical_conserved_canonical = canonical_conserved[~canonical_conserved[1].str.contains("canonical")]
canonical_conserved_canonical.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_ConservedAsCanonical.tsv"), header=None, index=None, sep="\t")
print("Conserved canonical human ORFs as canonical in other species")
print(len(set(canonical_conserved_canonical['orfID'].values.tolist())))

canonical_NonConserved = canonical[canonical[1].str.startswith("ENST")]
canonical_NonConserved.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_NonConserved.tsv"), header=None, index=None, sep="\t")
print("Non-Conserved canonical human ORFs")
print(len(set(canonical_NonConserved[0].values.tolist())))
canonical_NonConserved.columns = ['gene_name','gene_type','orfID',1]


all human
13368
all human after annotation
13368
noncoding all
1819
1590
Conserved noncoding human ORFs
86
Non-Conserved noncoding human ORFs
1590
Canonical all
11549
2547
Conserved canonical human ORFs
2132
Conserved canonical human ORFs as noncoding in other species
2107
Conserved canonical human ORFs as canonical in other species
244
Non-Conserved canonical human ORFs
2547


In [16]:
noncoding_conserved_to_matrix = noncoding_conserved
print(len(set(noncoding_conserved_to_matrix.orfID.values.tolist())))
# Assuming noncoding_conserved_to_matrix is already defined
noncoding_conserved_to_matrix['specie'] = noncoding_conserved_to_matrix[1].str.split("_", expand=True)[0]
noncoding_conserved_to_matrix = noncoding_conserved_to_matrix[['orfID', 'specie']]
noncoding_conserved_to_matrix['presence'] = 1

print(len(set(noncoding_conserved_to_matrix.orfID.values.tolist())))
noncoding_conserved_to_matrix

# Pivoting the data without creating an 'id', and keeping 'orfID' as the unique identifier
noncoding_conserved_matrix = noncoding_conserved_to_matrix.pivot_table(index='orfID', columns='specie', values='presence', aggfunc='max')

# Fill NaN values with 0
noncoding_conserved_matrix = noncoding_conserved_matrix.fillna(0)

# Reset index so 'orfID' becomes a column instead of an index
noncoding_conserved_matrix = noncoding_conserved_matrix.reset_index()

# Rename columns as necessary
noncoding_conserved_matrix.columns = ["orfID", "chicken", "macaca", "mouse", "opossum", "platypus"]

# Reorder columns for the final matrix
noncoding_conserved_matrix_ordered = noncoding_conserved_matrix[["orfID", "macaca", "mouse", "opossum", "platypus", "chicken"]]

# Calculate sum across species columns to identify how many species are present
noncoding_conserved_matrix_ordered['sum'] = noncoding_conserved_matrix_ordered.select_dtypes(include='number').sum(axis=1)

# Sort by the sum column to order by the number of species present
noncoding_conserved_matrix_ordered = noncoding_conserved_matrix_ordered.sort_values(by="sum", ascending=False)
print(len(set(noncoding_conserved_matrix_ordered.orfID.values.tolist())))

# Display the result
noncoding_conserved_matrix_ordered =  noncoding_conserved_matrix_ordered.merge(noncoding_conserved[['gene_name','gene_type','orfID']], on="orfID")
noncoding_conserved_matrix_ordered.drop_duplicates(inplace=True)

noncoding_conserved_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_Conserved_matrix.tsv"), sep="\t", index=None)
print(len(set(noncoding_conserved_matrix_ordered.orfID.values.tolist())))


86
86
86


/tmp/ipykernel_3596687/3091240473.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noncoding_conserved_to_matrix['specie'] = noncoding_conserved_to_matrix[1].str.split("_", expand=True)[0]
/tmp/ipykernel_3596687/3091240473.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noncoding_conserved_to_matrix['presence'] = 1
/tmp/ipykernel_3596687/3091240473.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

86


In [19]:
### all noncoding
noncoding_conserved_touse = noncoding_conserved
noncoding_conserved_touse['specie'] = noncoding_conserved_touse[1].str.split("_", expand=True)[0]
# noncoding_NonConserved.drop("specie",axis=1, inplace=True)
noncoding_to_matrix_raw = pd.concat([noncoding_conserved, noncoding_NonConserved])

noncoding_to_matrix = noncoding_to_matrix_raw[['orfID', 'specie']]
noncoding_to_matrix['presence'] = np.where(noncoding_to_matrix['specie'].isna(), 0, 1)


human = pd.DataFrame({
    'orfID': noncoding_to_matrix['orfID'].unique(),
    'specie': 'human',
    'presence': 1
})

noncoding_to_matrix = pd.concat([human, noncoding_to_matrix], ignore_index=True)
noncoding_to_matrix.dropna(inplace=True)

# Pivoting the data without creating an 'id', and keeping 'orfID' as the unique identifier
noncoding_matrix = noncoding_to_matrix.pivot_table(index='orfID', columns='specie', values='presence', aggfunc='max')
noncoding_matrix
# Fill NaN values with 0
noncoding_matrix = noncoding_matrix.fillna(0)

# Reset index so 'orfID' becomes a column instead of an index
noncoding_matrix = noncoding_matrix.reset_index()

# Rename columns as necessary
noncoding_matrix.columns = ["orfID", "chicken", "human", "macaca", "mouse", "opossum", "platypus"]

# Reorder columns for the final matrix
noncoding_matrix_ordered = noncoding_matrix[["orfID", "human","macaca", "mouse", "opossum", "platypus", "chicken"]]

# Calculate sum across species columns to identify how many species are present
noncoding_matrix_ordered['sum'] = noncoding_matrix_ordered.select_dtypes(include='number').sum(axis=1)

# Sort by the sum column to order by the number of species present
noncoding_matrix_ordered = noncoding_matrix_ordered.sort_values(by="sum", ascending=False)

# Display the result
noncoding_matrix_ordered =  noncoding_matrix_ordered.merge(noncoding_to_matrix_raw[['gene_name','gene_type','orfID']], on="orfID")
noncoding_matrix_ordered.drop_duplicates(inplace=True)

noncoding_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_matrix.tsv"), sep="\t", index=None)
print(len(set(noncoding_matrix_ordered.orfID.values.tolist())))

1590


/tmp/ipykernel_3596687/4161199752.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noncoding_conserved_touse['specie'] = noncoding_conserved_touse[1].str.split("_", expand=True)[0]
/tmp/ipykernel_3596687/4161199752.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noncoding_to_matrix['presence'] = np.where(noncoding_to_matrix['specie'].isna(), 0, 1)
/tmp/ipykernel_3596687/4161199752.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [43]:
canonical_conserved_to_matrix = canonical_conserved

# Assuming noncoding_conserved_to_matrix is already defined
canonical_conserved_to_matrix['specie'] = canonical_conserved_to_matrix[1].str.split("_", expand=True)[0]
canonical_conserved_to_matrix = canonical_conserved_to_matrix[['orfID', 'specie']]
canonical_conserved_to_matrix['presence'] = 1

# Pivoting the data without creating an 'id', and keeping 'orfID' as the unique identifier
canonical_conserved_matrix = canonical_conserved_to_matrix.pivot_table(index='orfID', columns='specie', values='presence', aggfunc='max')

# Fill NaN values with 0
canonical_conserved_matrix = canonical_conserved_matrix.fillna(0)

# Reset index so 'orfID' becomes a column instead of an index
canonical_conserved_matrix = canonical_conserved_matrix.reset_index()

# Rename columns as necessary
canonical_conserved_matrix.columns = ["orfID", "chicken", "macaca", "mouse", "opossum", "platypus"]

# Reorder columns for the final matrix
canonical_conserved_matrix_ordered = canonical_conserved_matrix[["orfID", "macaca", "mouse", "opossum", "platypus", "chicken"]]

# Calculate sum across species columns to identify how many species are present
canonical_conserved_matrix_ordered['sum'] = canonical_conserved_matrix_ordered.select_dtypes(include='number').sum(axis=1)

# Sort by the sum column to order by the number of species present
canonical_conserved_matrix_ordered = canonical_conserved_matrix_ordered.sort_values(by="sum", ascending=False)

# Display the result
canonical_conserved_matrix_ordered =  canonical_conserved_matrix_ordered.merge(canonical_conserved[['gene_name','gene_type','orfID']], on="orfID")
canonical_conserved_matrix_ordered.drop_duplicates(inplace=True)

canonical_conserved_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_Conserved_matrix.tsv"), sep="\t", index=None)

/tmp/ipykernel_3062604/2062637614.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canonical_conserved_to_matrix['specie'] = canonical_conserved_to_matrix[1].str.split("_", expand=True)[0]
/tmp/ipykernel_3062604/2062637614.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canonical_conserved_to_matrix['presence'] = 1
/tmp/ipykernel_3062604/2062637614.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [44]:
### all canonical
canonical_conserved_touse = canonical_conserved
canonical_conserved_touse['specie'] = canonical_conserved_touse[1].str.split("_", expand=True)[0]
# noncoding_NonConserved.drop("specie",axis=1, inplace=True)
canonical_to_matrix_raw = pd.concat([canonical_conserved, canonical_NonConserved])

canonical_to_matrix = canonical_to_matrix_raw[['orfID', 'specie']]
canonical_to_matrix['presence'] = np.where(canonical_to_matrix['specie'].isna(), 0, 1)


human = pd.DataFrame({
    'orfID': canonical_to_matrix['orfID'].unique(),
    'specie': 'human',
    'presence': 1
})

canonical_to_matrix = pd.concat([human, canonical_to_matrix], ignore_index=True)
canonical_to_matrix.dropna(inplace=True)

# Pivoting the data without creating an 'id', and keeping 'orfID' as the unique identifier
canonical_matrix = canonical_to_matrix.pivot_table(index='orfID', columns='specie', values='presence', aggfunc='max')
canonical_matrix
# Fill NaN values with 0
canonical_matrix = canonical_matrix.fillna(0)

# Reset index so 'orfID' becomes a column instead of an index
canonical_matrix = canonical_matrix.reset_index()

# Rename columns as necessary
canonical_matrix.columns = ["orfID", "chicken", "human", "macaca", "mouse", "opossum", "platypus"]

# Reorder columns for the final matrix
canonical_matrix_ordered = canonical_matrix[["orfID", "human","macaca", "mouse", "opossum", "platypus", "chicken"]]

# Calculate sum across species columns to identify how many species are present
canonical_matrix_ordered['sum'] = canonical_matrix_ordered.select_dtypes(include='number').sum(axis=1)

# Sort by the sum column to order by the number of species present
canonical_matrix_ordered = canonical_matrix_ordered.sort_values(by="sum", ascending=False)

# Display the result
canonical_matrix_ordered =  canonical_matrix_ordered.merge(canonical_to_matrix_raw[['gene_name','gene_type','orfID']], on="orfID")
canonical_matrix_ordered.drop_duplicates(inplace=True)

canonical_matrix_ordered.to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_matrix.tsv"), sep="\t", index=None)

/tmp/ipykernel_3062604/2806917182.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canonical_conserved_touse['specie'] = canonical_conserved_touse[1].str.split("_", expand=True)[0]
/tmp/ipykernel_3062604/2806917182.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canonical_to_matrix['presence'] = np.where(canonical_to_matrix['specie'].isna(), 0, 1)
/tmp/ipykernel_3062604/2806917182.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Find the oldest specie containg the ORF translated

In [16]:
species_ordered = ['chicken','platypus','opossum','mouse','macaca','human']
def find_last_species(row):
    for sp in species_ordered:
        if row[sp] == 1.0:  # Check if the value is 1.0
            return sp
    return None  # Return None if no species has a 1

In [45]:
# Apply the function to each row to create a new column 'oldest_species'
canonical_matrix_ordered['oldest_species'] = canonical_matrix_ordered.apply(find_last_species, axis=1)
canonical_matrix_ordered = species_and_years.merge(canonical_matrix_ordered, left_on=["specie"], right_on="oldest_species")
canonical_matrix_ordered[['orfID','gene_name','gene_type','oldest_species','age']].to_csv(os.path.join(evoDir,"Clustering/canonicalHuman_matrix_oldestAge.tsv"), sep="\t", index=None)

noncoding_matrix_ordered['oldest_species'] = noncoding_matrix_ordered.apply(find_last_species, axis=1)
noncoding_matrix_ordered = species_and_years.merge(noncoding_matrix_ordered, left_on=["specie"], right_on="oldest_species")
noncoding_matrix_ordered[['orfID','gene_name','gene_type','oldest_species','age']].to_csv(os.path.join(evoDir,"Clustering/noncodingHuman_matrix_oldestAge.tsv"), sep="\t", index=None)

complete_matrix_age = pd.concat([noncoding_matrix_ordered[['orfID','gene_name','gene_type','oldest_species','age']], canonical_matrix_ordered[['orfID','gene_name','gene_type','oldest_species','age']]])
complete_matrix_age.to_csv(os.path.join(evoDir,"Clustering/allHuman_matrix_oldestAge.tsv"), sep="\t", index=None)
